# Module 4: Hourly Updates

## Purpose
This module runs hourly to monitor and adjust prices based on:
1. **UTH (Up-Till-Hour) Performance**: Cumulative qty/retailers from start of day until current hour
2. **Last Hour Performance**: Qty/retailers for the most recent hour only

## Schedule
- Runs hourly from 12 PM to 12 AM (midnight), **except** Module 3 hours (12 PM, 3 PM, 6 PM, 9 PM)
- Also runs once at 3 AM
- Active hours: 1 PM, 2 PM, 4 PM, 5 PM, 7 PM, 8 PM, 10 PM, 11 PM, 12 AM, 3 AM

## Data Flow
```
data_extraction.ipynb → Snowflake (Pricing_data_extraction)
                              ↓
                        Module 4 (this module)
                           ├── Load p80_qty, p70_retailers, std columns
                           ├── Fetch fresh: cart rules, stocks, WAC
                           ├── Calculate new_wac (from today's PRS)
                           ├── Query UTH performance (today)
                           ├── Query Last Hour performance (today)
                           ├── Query historical hour contributions
                           ├── Calculate targets & statuses (±3 std)
                           └── Generate actions (TBD)
```

## New WAC Calculation
To avoid database lag, we calculate a fresh WAC from today's purchase receipts:
- `new_wac1` = ((stocks + reflected_qty) × wac1 + unreflected_qty × item_price) / total_qty
- `new_wac_p` = wac_p × (1 + wac1_change)
- `new_wac` = new_wac_p (with fallback to current_wac)

## Status Logic (±3 Standard Deviations)
SKU status is determined by comparing actual performance to target ± 3 std:
- **Growing**: actual > target + 3×std
- **On Track**: target - 3×std ≤ actual ≤ target + 3×std
- **Dropping**: actual < target - 3×std (minimum threshold = 1)

Standard deviation columns used:
- Qty: `std_daily_240d`
- Retailers: `std_daily_retailers_240d`

## Status Outputs
- `uth_qty_status`: growing / dropping / on_track
- `uth_rets_status`: growing / dropping / on_track
- `last_hour_qty_status`: growing / dropping / on_track
- `last_hour_rets_status`: growing / dropping / on_track


In [1]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import sys
sys.path.append('..')
import setup_environment_2
# Import queries module for Snowflake access
%run queries_module.ipynb

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
CURRENT_HOUR = CAIRO_NOW.hour

print(f"Module 4: Hourly Updates")
print(f"Current Cairo Time: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current Hour: {CURRENT_HOUR}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


Queries Module | Timezone: America/Los_Angeles
✅ UTH and Last Hour functions defined

QUERIES MODULE READY

Live Data Functions:
  • get_current_stocks()
  • get_packing_units()
  • get_current_prices()
  • get_current_wac()
  • get_current_cart_rules()

UTH Performance Functions:
  • get_uth_performance()         - UTH qty/retailers (Snowflake)
  • get_hourly_distribution()     - Historical hour contributions (Snowflake)
  • get_last_hour_performance()   - Last hour qty/retailers (DWH)

Note: Market prices use MODULE_1_INPUT data
Retailer Selection Queries defined ✓
  - get_churned_dropped_retailers()
  - get_category_not_product_retailers()
  - get_out_of_cycle_retailers()
  - get_view_no_orders_retailers()
  - get_excluded_retailers()
  - get_retailers_with_quantity_discount()
  - get_retailer_main_warehouse()
Module 4: Hourly Updates
Current Cairo Time: 2026-02-01 13:01:38
Current Hour: 13


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Input/Output configuration
# Data is now loaded from Snowflake instead of Excel
INPUT_TABLE = 'MATERIALIZED_VIEWS.Pricing_data_extraction'
OUTPUT_FILE = f'module_4_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'

# Status thresholds (±3 std from target)
# - Growing: actual > target + 3*std
# - On Track: target - 3*std <= actual <= target + 3*std
# - Dropping: actual < target - 3*std (minimum threshold = 1)
STD_THRESHOLD = 3  # Number of standard deviations
MIN_DROPPING_THRESHOLD = 1  # Minimum threshold for dropping status

# Module 3 hours (skip these)
MODULE_3_HOURS = [12, 15, 18, 21]  # 12 PM, 3 PM, 6 PM, 9 PM

print(f"Input: {INPUT_TABLE} (today's data)")
print(f"Output: {OUTPUT_FILE}")
print(f"Status Thresholds: ±{STD_THRESHOLD} std (Dropping minimum = {MIN_DROPPING_THRESHOLD})")


Input: MATERIALIZED_VIEWS.Pricing_data_extraction (today's data)
Output: module_4_output_20260201_1301.xlsx
Status Thresholds: ±3 std (Dropping minimum = 1)


In [3]:
# =============================================================================
# LOAD DATA FROM SNOWFLAKE (Instead of Excel file)
# =============================================================================
print("Loading data from Snowflake...")

# Query to get today's data from Pricing_data_extraction
LOAD_QUERY = f"""
SELECT distinct * FROM {INPUT_TABLE}
WHERE created_at = '{datetime.now(CAIRO_TZ).date()}'
"""

df = query_snowflake(LOAD_QUERY)
print(f"Loaded {len(df)} records from Snowflake")

# Ensure required columns exist with proper types
df['p80_daily_240d'] = pd.to_numeric(df.get('p80_daily_240d', 0), errors='coerce').fillna(0)
df['p70_daily_retailers_240d'] = pd.to_numeric(df.get('p70_daily_retailers_240d', 1), errors='coerce').fillna(1)
df['std_daily_240d'] = pd.to_numeric(df.get('std_daily_240d', 0), errors='coerce').fillna(0)
df['std_daily_retailers_240d'] = pd.to_numeric(df.get('std_daily_retailers_240d', 0), errors='coerce').fillna(0)
df['warehouse_id'] = df['warehouse_id'].astype(int)
df['product_id'] = df['product_id'].astype(int)
df['cohort_id'] = df['cohort_id'].astype(int) if 'cohort_id' in df.columns else None
df['commercial_min_price'] = pd.to_numeric(df.get('commercial_min_price', 0), errors='coerce')
df['commercial_min_price'] = np.round(df['commercial_min_price']*4)/4
# Get category for hourly distribution merge
if 'cat' not in df.columns and 'category' in df.columns:
    df['cat'] = df['category']

print(f"\nP80 Qty Stats: min={df['p80_daily_240d'].min():.1f}, max={df['p80_daily_240d'].max():.1f}, mean={df['p80_daily_240d'].mean():.1f}")
print(f"P70 Retailers Stats: min={df['p70_daily_retailers_240d'].min():.1f}, max={df['p70_daily_retailers_240d'].max():.1f}, mean={df['p70_daily_retailers_240d'].mean():.1f}")
print(f"Std Qty Stats: min={df['std_daily_240d'].min():.1f}, max={df['std_daily_240d'].max():.1f}, mean={df['std_daily_240d'].mean():.1f}")
print(f"Std Retailers Stats: min={df['std_daily_retailers_240d'].min():.1f}, max={df['std_daily_retailers_240d'].max():.1f}, mean={df['std_daily_retailers_240d'].mean():.1f}")

# =============================================================================
# GET FRESH DATA FROM QUERIES MODULE (Snowflake)
# =============================================================================

# 1. Current Cart Rules
df_cart_rules = get_current_cart_rules()

# Merge with main df (by cohort_id + product_id)
if 'cohort_id' in df.columns and len(df_cart_rules) > 0:
    df = df.drop(columns=['current_cart_rule'], errors='ignore')
    df = df.merge(df_cart_rules, on=['cohort_id', 'product_id'], how='left')
    df['current_cart_rule'] = df['current_cart_rule'].fillna(999)
    print(f"✅ Merged cart rules: {len(df)} records")
else:
    df['current_cart_rule'] = df.get('current_cart_rule', 999)

# 2. Current Stocks
df_stocks = get_current_stocks()

# Merge stocks (by warehouse_id + product_id)
if len(df_stocks) > 0:
    df = df.drop(columns=['stocks'], errors='ignore')
    df = df.merge(df_stocks, on=['warehouse_id', 'product_id'], how='left')
    df['stocks'] = df['stocks'].fillna(0)
    print(f"✅ Merged stocks: {len(df)} records")
else:
    df['stocks'] = df.get('stocks', 0)

# 3. Current WAC (Weighted Average Cost)
df_wac = get_current_wac()

# Merge WAC (by warehouse_id + product_id)
if len(df_wac) > 0:
    df = df.drop(columns=['wac_p'], errors='ignore')
    df = df.merge(df_wac, on=['product_id'], how='left')
    df['wac_p'] = df['wac_p'].fillna(0)
    print(f"✅ Merged WAC: {len(df)} records")
else:
    df['wac_p'] = df.get('wac_p', 0)

# 4. Current Prices
df_prices = get_current_prices()

# Merge prices (by cohort_id + product_id)
if len(df_prices) > 0:
    df = df.drop(columns=['current_price'], errors='ignore')
    df = df.merge(df_prices[['cohort_id', 'product_id', 'current_price']], on=['cohort_id', 'product_id'], how='left')
    df['current_price'] = df['current_price'].fillna(0)
    print(f"✅ Merged current prices: {len(df)} records")
else:
    df['current_price'] = df.get('current_price', 0)

print(f"\nCurrent Stock Stats: min={df['stocks'].min():.0f}, max={df['stocks'].max():.0f}, mean={df['stocks'].mean():.1f}")
print(f"Current WAC Stats: min={df['wac_p'].min():.2f}, max={df['wac_p'].max():.2f}, mean={df['wac_p'].mean():.2f}")
print(f"Current Price Stats: min={df['current_price'].min():.2f}, max={df['current_price'].max():.2f}, mean={df['current_price'].mean():.2f}")


Loading data from Snowflake...


Loaded 28382 records from Snowflake

P80 Qty Stats: min=5.0, max=1652.8, mean=12.6
P70 Retailers Stats: min=2.0, max=154.0, mean=3.6
Std Qty Stats: min=0.0, max=2574.8, mean=6.2
Std Retailers Stats: min=0.0, max=75.8, mean=1.4
Fetching current cart rules...


  Loaded 72968 records
✅ Merged cart rules: 28382 records
Fetching current stocks...


  Loaded 1855303 records


✅ Merged stocks: 28382 records
Fetching current WAC...


  Loaded 8153 records
✅ Merged WAC: 28382 records
Fetching current prices...


  Loaded 116397 records
✅ Merged current prices: 28390 records

Current Stock Stats: min=0, max=26733, mean=57.5
Current WAC Stats: min=0.01, max=2082.02, mean=227.96
Current Price Stats: min=0.01, max=2208.25, mean=238.54


In [4]:
# =============================================================================
# CALCULATE NEW WAC (Accounting for Today's Unreflected Purchases)
# =============================================================================
# This calculates a fresh WAC that accounts for today's purchase receipts
# that haven't been reflected in the database yet (to avoid lag)

print("Calculating new WAC from today's purchase receipts...")

# 1. Query today's PRS data (purchases by product_id)
prs_query = '''
WITH prs AS (
    SELECT DISTINCT 
        product_purchased_receipts.purchased_receipt_id,
        products.id AS product_id,
        product_purchased_receipts.basic_unit_count,
        product_purchased_receipts.purchased_item_count * product_purchased_receipts.basic_unit_count AS purchase_min_count,
        product_purchased_receipts.final_price / product_purchased_receipts.purchased_item_count AS final_item_price
    FROM product_purchased_receipts
    LEFT JOIN products ON products.id = product_purchased_receipts.product_id
    LEFT JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
    WHERE product_purchased_receipts.purchased_item_count <> 0
        AND purchased_receipts.purchased_receipt_status_id IN (4,5,7)
        AND purchased_receipts.date::date >= current_date
        AND product_purchased_receipts.final_price > 0 
)
SELECT 
    product_id,
    SUM(purchase_min_count) AS all_day_qty,
    AVG(final_item_price / basic_unit_count) AS item_price
FROM prs
GROUP BY 1
'''
df_prs = setup_environment_2.dwh_pg_query(prs_query, columns=['product_id', 'all_day_qty', 'item_price'])
try:
    df_prs['product_id'] = pd.to_numeric(df_prs['product_id'])
    df_prs['all_day_qty'] = pd.to_numeric(df_prs['all_day_qty'])
    df_prs['item_price'] = pd.to_numeric(df_prs['item_price'])
except:
    df_prs=pd.DataFrame(columns=['product_id', 'all_day_qty', 'item_price'])
print(f"  Loaded {len(df_prs)} PRS records (today's purchases)")

# 2. Query what's already reflected in WAC tracker
reflected_query = '''
SELECT
    t_product_id AS product_id,
    s_beg_stock AS av_stocks,
    p_purchased_item_count AS pr_qty
FROM finance.wac_tracker wt
WHERE wt.t_date::date = CURRENT_DATE
    AND p_purchased_item_count > 0 
'''
try:
    df_reflected = setup_environment_2.dwh_pg_query(reflected_query, columns=['product_id', 'av_stocks', 'pr_qty'])
    df_reflected['product_id'] = pd.to_numeric(df_reflected['product_id'])
    df_reflected['av_stocks'] = pd.to_numeric(df_reflected['av_stocks'])
    df_reflected['pr_qty'] = pd.to_numeric(df_reflected['pr_qty'])
    print(f"  Loaded {len(df_reflected)} WAC tracker records (already reflected)")
except:
    df_reflected = pd.DataFrame(columns=['product_id', 'av_stocks', 'pr_qty'])
    print("  No WAC tracker records found for today")

# 3. Query base WAC (wac1, wac_p) from all_cogs
wac_base_query = '''
SELECT 
    f.product_id,
    f.wac1,
    f.wac_p
FROM finance.all_cogs f
JOIN products ON products.id = f.product_id
JOIN categories ON products.category_id = categories.id
WHERE current_timestamp BETWEEN f.from_date AND f.to_date 
    AND NOT categories.name_ar IN (
        SELECT categories.name_ar AS cat
        FROM categories
        JOIN sections s ON s.id = categories.section_id
        WHERE categories.name_ar LIKE '%سايب%'
            OR categories.name_ar LIKE '%بالتة%'
            OR categories.section_id IN (225, 318, 285, 121, 87, 351, 417)
    )
'''
df_wac_base = query_snowflake(wac_base_query)
df_wac_base['product_id'] = pd.to_numeric(df_wac_base['product_id'])
df_wac_base['wac1'] = pd.to_numeric(df_wac_base['wac1'])
df_wac_base['wac_p'] = pd.to_numeric(df_wac_base['wac_p'])
print(f"  Loaded {len(df_wac_base)} base WAC records from all_cogs")

# 4. Merge and calculate new WAC
df_wac_calc = df_wac_base.merge(df_prs, on='product_id', how='left')
df_wac_calc = df_wac_calc.merge(df_reflected, on='product_id', how='left')
df_wac_calc = df_wac_calc.fillna(0)

# Use current_stock from main df if av_stocks is 0
df_stock_lookup = df[['product_id', 'stocks']].drop_duplicates()
df_stock_lookup = df_stock_lookup.groupby(['product_id'])['stocks'].sum().reset_index()
df_wac_calc = df_wac_calc.merge(df_stock_lookup, on='product_id', how='left')
df_wac_calc['av_stocks'] = df_wac_calc.apply(
    lambda row: row['stocks'] if row['av_stocks'] == 0 else row['av_stocks'], axis=1)

# Calculate not reflected qty (purchases not yet in WAC tracker)
df_wac_calc['not_reflected_qty'] = df_wac_calc['all_day_qty'] - df_wac_calc['pr_qty']
df_wac_calc['not_reflected_qty'] = df_wac_calc['not_reflected_qty'].clip(lower=0)  # Can't be negative

# Calculate new WAC
# new_wac1 = ((stocks + pr_qty) * wac1 + not_reflected_qty * item_price) / (stocks + pr_qty + not_reflected_qty)
denominator = df_wac_calc['av_stocks'] + df_wac_calc['pr_qty'] + df_wac_calc['not_reflected_qty']
numerator = ((df_wac_calc['av_stocks'] + df_wac_calc['pr_qty']) * df_wac_calc['wac1'] + 
             df_wac_calc['not_reflected_qty'] * df_wac_calc['item_price'])

df_wac_calc['new_wac1'] = np.where(denominator > 0, numerator / denominator, df_wac_calc['wac1'])

# Calculate wac1 change and new_wac_p
df_wac_calc['wac1_change'] = np.where(
    df_wac_calc['wac1'] > 0,
    (df_wac_calc['new_wac1'] - df_wac_calc['wac1']) / df_wac_calc['wac1'],
    0
)
df_wac_calc['new_wac_p'] = df_wac_calc['wac_p'] * (1 + df_wac_calc['wac1_change'])
df_wac_calc['new_wac_p'] = df_wac_calc['new_wac_p'].fillna(df_wac_calc['wac_p'])

# Prepare final new_wac dataframe
df_new_wac = df_wac_calc[['product_id','new_wac_p', 'not_reflected_qty']].copy()
df_new_wac=df_new_wac.drop_duplicates()

# 5. Merge new_wac into main dataframe
df = df.drop(columns=['new_wac_p','not_reflected_qty'], errors='ignore')
df = df.merge(df_new_wac, on='product_id', how='left')
df['new_wac'] = df['new_wac_p'].fillna(df['wac_p'])  # Fallback to current_wac if no new_wac

print(f"\n✅ New WAC calculated: {len(df)} records")
print(f"   Products with unreflected purchases: {(df['not_reflected_qty'] > 0).sum()}")
print(f"   New WAC Stats: min={df['new_wac'].min():.2f}, max={df['new_wac'].max():.2f}, mean={df['new_wac'].mean():.2f}")


Calculating new WAC from today's purchase receipts...


  Loaded 86 PRS records (today's purchases)
Shape of passed values is (0, 1), indices imply (0, 3)
  No WAC tracker records found for today


  Loaded 7693 base WAC records from all_cogs

✅ New WAC calculated: 28390 records
   Products with unreflected purchases: 874
   New WAC Stats: min=0.01, max=2082.02, mean=227.96


/tmp/ipykernel_6519/2942681164.py:90: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_wac_calc = df_wac_calc.fillna(0)


In [5]:
# =============================================================================
# QUERY 1: TODAY'S UTH (Up-Till-Hour) PERFORMANCE
# =============================================================================
# Gets cumulative qty and retailers from start of day until current hour
# Uses get_uth_performance() from queries_module

df_uth_today = get_uth_performance()


Fetching UTH performance from Snowflake...


  Loaded 6405 UTH records


In [6]:
# =============================================================================
# QUERY 2: TODAY'S LAST HOUR PERFORMANCE (from DWH)
# =============================================================================
# Gets qty and retailers for the PREVIOUS hour only (not cumulative)
# Uses get_last_hour_performance() from queries_module (DWH/PostgreSQL)

df_last_hour = get_last_hour_performance()


Fetching last hour performance from DWH...


  Loaded 1055 last hour records from DWH


In [7]:
# =============================================================================
# QUERY 3: HISTORICAL HOURLY DISTRIBUTION (Last 4 Months) - By Category & Warehouse
# =============================================================================
# Gets:
# - avg_uth_pct_qty/retailers: Average contribution of hours 0 to (current_hour-1) to daily total
# - avg_last_hour_pct_qty/retailers: Average contribution of (current_hour-1) alone to daily total
# Uses get_hourly_distribution() from queries_module

df_hourly_dist = get_hourly_distribution()
print(f"\nAvg UTH % (qty): {df_hourly_dist['avg_uth_pct_qty'].mean()*100:.1f}%")
print(f"Avg Last Hour % (qty): {df_hourly_dist['avg_last_hour_pct_qty'].mean()*100:.1f}%")


Fetching hourly distribution from Snowflake...


  Loaded 771 hourly distribution records

Avg UTH % (qty): 31.9%
Avg Last Hour % (qty): 3.8%


In [8]:
# =============================================================================
# MERGE DATA
# =============================================================================
print("Merging performance data with base data...")

# Merge UTH today data
if len(df_uth_today) > 0:
    df = df.merge(df_uth_today, on=['warehouse_id', 'product_id'], how='left')
else:
    df['uth_qty'] = 0
    df['uth_nmv'] = 0
    df['uth_retailers'] = 0

# Merge last hour data
if len(df_last_hour) > 0:
    df = df.merge(df_last_hour, on=['warehouse_id', 'product_id'], how='left')
else:
    df['last_hour_qty'] = 0
    df['last_hour_nmv'] = 0
    df['last_hour_retailers'] = 0

# Merge hourly distribution (by warehouse_id + cat)
if len(df_hourly_dist) > 0:
    df = df.merge(df_hourly_dist, on=['warehouse_id', 'cat'], how='left')
else:
    df['avg_uth_pct_qty'] = 0.5
    df['avg_uth_pct_retailers'] = 0.5
    df['avg_last_hour_pct_qty'] = 0.05
    df['avg_last_hour_pct_retailers'] = 0.05

# Fill NaN values
df['uth_qty'] = df['uth_qty'].fillna(0)
df['uth_nmv'] = df['uth_nmv'].fillna(0)
df['uth_retailers'] = df['uth_retailers'].fillna(0)
df['last_hour_qty'] = df['last_hour_qty'].fillna(0)
df['last_hour_nmv'] = df['last_hour_nmv'].fillna(0)
df['last_hour_retailers'] = df['last_hour_retailers'].fillna(0)
df['avg_uth_pct_qty'] = df['avg_uth_pct_qty'].fillna(0.5)
df['avg_uth_pct_retailers'] = df['avg_uth_pct_retailers'].fillna(0.5)
df['avg_last_hour_pct_qty'] = df['avg_last_hour_pct_qty'].fillna(0.05)
df['avg_last_hour_pct_retailers'] = df['avg_last_hour_pct_retailers'].fillna(0.05)

print(f"✅ Merged data: {len(df)} records")
print(f"\nUTH Qty Stats: min={df['uth_qty'].min():.0f}, max={df['uth_qty'].max():.0f}, mean={df['uth_qty'].mean():.1f}")
print(f"Last Hour Qty Stats: min={df['last_hour_qty'].min():.0f}, max={df['last_hour_qty'].max():.0f}, mean={df['last_hour_qty'].mean():.1f}")
print(f"Current Cart Rule Stats: min={df['current_cart_rule'].min():.0f}, max={df['current_cart_rule'].max():.0f}, mean={df['current_cart_rule'].mean():.1f}")


Merging performance data with base data...
✅ Merged data: 28390 records

UTH Qty Stats: min=0, max=432, mean=1.4
Last Hour Qty Stats: min=0, max=44, mean=0.1
Current Cart Rule Stats: min=2, max=1820, mean=19.8


In [9]:
# =============================================================================
# CALCULATE TARGETS AND STATUSES
# =============================================================================
print("Calculating UTH and Last Hour targets and statuses...")

def get_status_std(actual, target, std, is_qty=True):
    """
    Determine status based on ±3 std from target.
    - Growing: actual > target + 3*std
    - On Track: target - 3*std <= actual <= target + 3*std
    - Dropping: actual < target - 3*std (minimum threshold = 1)
    
    Parameters:
    - actual: actual performance value
    - target: target value (p80 for qty, p70 for retailers)
    - std: standard deviation (std_daily_240d or std_daily_retailers_240d)
    - is_qty: True for qty metrics, False for retailer metrics
    """
    upper_bound = target + STD_THRESHOLD * std
    lower_bound = max(target - STD_THRESHOLD * std, MIN_DROPPING_THRESHOLD)
    
    if actual > upper_bound:
        return 'growing'
    elif actual < lower_bound:
        return 'dropping'
    else:
        return 'on_track'

# Calculate UTH targets
# UTH target = p80_qty * avg_uth_pct (historical % of day that should be done by now)
df['uth_qty_target'] = df['p80_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate UTH std thresholds (scaled by UTH percentage)
df['uth_qty_std'] = df['std_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_std'] = df['std_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate Last Hour targets
# Last hour target = p80_qty * avg_last_hour_pct (historical % of day for this hour)
df['last_hour_qty_target'] = df['p80_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate Last Hour std thresholds (scaled by last hour percentage)
df['last_hour_qty_std'] = df['std_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_std'] = df['std_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate statuses using ±3 std thresholds
df['uth_qty_status'] = df.apply(
    lambda row: get_status_std(row['uth_qty'], row['uth_qty_target'], row['uth_qty_std'], is_qty=True), axis=1)
df['uth_rets_status'] = df.apply(
    lambda row: get_status_std(row['uth_retailers'], row['uth_rets_target'], row['uth_rets_std'], is_qty=False), axis=1)
df['last_hour_qty_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_qty'], row['last_hour_qty_target'], row['last_hour_qty_std'], is_qty=True), axis=1)
df['last_hour_rets_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_retailers'], row['last_hour_rets_target'], row['last_hour_rets_std'], is_qty=False), axis=1)

print(f"✅ Targets and statuses calculated using ±{STD_THRESHOLD} std thresholds")

# Summary
print(f"\n{'='*60}")
print("UTH STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nUTH Qty Status:")
print(df['uth_qty_status'].value_counts().to_string())
print(f"\nUTH Retailers Status:")
print(df['uth_rets_status'].value_counts().to_string())

print(f"\n{'='*60}")
print("LAST HOUR STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nLast Hour Qty Status:")
print(df['last_hour_qty_status'].value_counts().to_string())
print(f"\nLast Hour Retailers Status:")
print(df['last_hour_rets_status'].value_counts().to_string())


Calculating UTH and Last Hour targets and statuses...


✅ Targets and statuses calculated using ±3 std thresholds

UTH STATUS DISTRIBUTION

UTH Qty Status:
uth_qty_status
dropping    22528
on_track     5135
growing       727

UTH Retailers Status:
uth_rets_status
dropping    22279
on_track     4811
growing      1300

LAST HOUR STATUS DISTRIBUTION

Last Hour Qty Status:
last_hour_qty_status
dropping    27388
growing       519
on_track      483

Last Hour Retailers Status:
last_hour_rets_status
dropping    27388
growing       693
on_track      309


In [10]:
# =============================================================================
# FETCH MARKET DATA AND MARGIN TIERS
# =============================================================================
print("Fetching market data and margin tiers...")

# Import market_data_module for get_market_data()
%run market_data_module.ipynb

# 1. Get margin tiers (already available from queries_module)
df_margin_tiers = get_margin_tiers()
print(f"✅ Loaded {len(df_margin_tiers)} margin tier records")

# 2. Get market data
df_market = get_market_data()
print(f"✅ Loaded {len(df_market)} market data records")

# Verify cohort_id is available in all dataframes
print(f"\nChecking cohort_id availability:")
print(f"  - df has cohort_id: {'cohort_id' in df.columns}")
print(f"  - df_margin_tiers has cohort_id: {'cohort_id' in df_margin_tiers.columns}")
print(f"  - df_market has cohort_id: {'cohort_id' in df_market.columns}")

# 3. Merge margin tiers with df (by cohort_id + product_id) - ALL COLUMNS
merge_keys = ['cohort_id', 'product_id']
df = df.drop(columns=[c for c in df_margin_tiers.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_margin_tiers, on=merge_keys, how='left')
print(f"\n✅ Merged margin tiers: {len(df)} records")
print(f"   Margin tier columns added: {[c for c in df_margin_tiers.columns if c not in merge_keys]}")

# 4. Merge market data with df (by cohort_id + product_id) - ALL COLUMNS
df = df.drop(columns=[c for c in df_market.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_market, on=merge_keys, how='left')
print(f"✅ Merged market data: {len(df)} records")
print(f"   Market columns added: {[c for c in df_market.columns if c not in merge_keys]}")

# 5. Calculate current margin based on current_price and new_wac
df['current_margin'] = (df['current_price'] - df['new_wac']) / df['current_price']
df['current_margin'] = df['current_margin'].fillna(0)

print(f"\nMargin Stats: min={df['current_margin'].min():.2%}, max={df['current_margin'].max():.2%}, mean={df['current_margin'].mean():.2%}")

# =============================================================================
# FILTER SKUS FOR ACTION
# =============================================================================
# Filter SKUs that need action based on:
# Pre-condition: below_min_stock_flag != 1 (must be sellable)
# Condition A: new_wac > wac_p * 1.005 (WAC increased by more than 0.5%)
# Condition B: Last hour growing (qty OR rets) AND UTH (one growing + other on_track/growing) AND last_hour_qty > 5

print("Filtering SKUs for action...")

# Skip SKUs with below_min_stock_flag (stock < min selling unit qty)
can_sell = df['below_min_stock_flag'].fillna(0) != 1

# Condition A: WAC increased significantly
condition_a = (df['new_wac'] > df['wac_p'] * 1.005)

# Condition B: Last hour growing AND UTH performing well AND last_hour_qty > 5
# UTH good = one must be 'growing' AND the other must be 'on_track' or 'growing'
last_hour_growing = (df['last_hour_qty_status'] == 'growing') | (df['last_hour_rets_status'] == 'growing')
uth_good = (
    ((df['uth_qty_status'] == 'growing') & (df['uth_rets_status'].isin(['on_track', 'growing']))) |
    ((df['uth_rets_status'] == 'growing') & (df['uth_qty_status'].isin(['on_track', 'growing'])))
)
last_hour_qty_threshold = df['last_hour_qty'] > 5
condition_b = last_hour_growing & uth_good & last_hour_qty_threshold & can_sell

# Condition C: Current price below commercial minimum (must enforce commercial min)
condition_c = (
    (df['commercial_min_price'].notna()) & 
    (df['commercial_min_price'] > 0) & 
    (df['current_price'] < df['commercial_min_price'])
)

# Combine conditions (A OR B OR C)
df['needs_action'] = condition_a | condition_b | condition_c
df['action_reason'] = 'none'
df.loc[condition_a & ~condition_b & ~condition_c, 'action_reason'] = 'wac_increase'
df.loc[~condition_a & condition_b & ~condition_c, 'action_reason'] = 'growing_performance'
df.loc[~condition_a & ~condition_b & condition_c, 'action_reason'] = 'below_commercial_min'
df.loc[condition_a & condition_b & ~condition_c, 'action_reason'] = 'both'
df.loc[condition_a & ~condition_b & condition_c, 'action_reason'] = 'wac_and_commercial'
df.loc[~condition_a & condition_b & condition_c, 'action_reason'] = 'growing_and_commercial'
df.loc[condition_a & condition_b & condition_c, 'action_reason'] = 'all'

# Define important columns for df_action
action_columns = [
    # Identification
    'warehouse_id', 'product_id', 'cohort_id', 'sku', 'region', 'brand', 'cat',
    # Price & Cost
    'current_price', 'wac_p', 'new_wac', 'current_margin', 'commercial_min_price',
    # Inventory & Rules
    'stocks', 'current_cart_rule',
    # Performance Targets
    'p80_daily_240d', 'p70_daily_retailers_240d', 'std_daily_240d', 'std_daily_retailers_240d',
    # Refill columns for cart rule calculation
    'normal_refill', 'refill_stddev', 'target_margin',
    # UTH Status
    'uth_qty', 'uth_qty_status', 'uth_retailers', 'uth_rets_status',
    # Last Hour Status
    'last_hour_qty', 'last_hour_qty_status', 'last_hour_retailers', 'last_hour_rets_status',
    # Market Margins
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market',
    # Margin Tiers (excluding: effective_min_margin, max_boundary, margin_step, margin_tier_below)
    'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
    'margin_tier_above_1', 'margin_tier_above_2',
    # Action Flags
    'needs_action', 'action_reason'
]

# Filter to only columns that exist in df
action_columns = [c for c in action_columns if c in df.columns]

# Filter to action SKUs with selected columns only
df_action = df[df['needs_action']][action_columns].copy()

print(f"\n{'='*60}")
print("SKUs FILTERED FOR ACTION")
print(f"{'='*60}")
print(f"\nTotal SKUs needing action: {len(df_action)} out of {len(df)} ({len(df_action)/len(df)*100:.1f}%)")
print(f"\nBreakdown by reason:")
print(df_action['action_reason'].value_counts().to_string())

print(f"\n--- Condition A (WAC increase > 0.5%): {condition_a.sum()} SKUs")
print(f"--- Condition B (Last hour growing + UTH good + qty > 5): {condition_b.sum()} SKUs")
print(f"--- Condition C (Below commercial minimum): {condition_c.sum()} SKUs")

# Show sample
print(f"\n{'='*60}")
print("SAMPLE ACTION SKUs")
print(f"{'='*60}")
print(f"\nTotal columns in df_action: {len(df_action.columns)}")
print(f"Columns: {df_action.columns.tolist()}")
display(df_action.head(10))


Fetching market data and margin tiers...


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


Market Data Module loaded at 2026-02-01 13:02:26 Cairo time
Snowflake timezone: America/Los_Angeles
All queries defined ✓
Helper functions defined ✓
get_market_data() function defined ✓
get_margin_tiers() function defined ✓

MARKET DATA MODULE READY

Available functions (NO INPUT REQUIRED):
  - get_market_data()   : Fetch and process all market prices
  - get_margin_tiers()  : Fetch and calculate margin tiers

Usage:
  %run market_data_module.ipynb
  df_market = get_market_data()
  df_tiers = get_margin_tiers()

FETCHING MARGIN TIERS
Timestamp: 2026-02-01 13:02:27 Cairo time

Step 1: Fetching margin boundaries from PRODUCT_STATISTICS...


    Loaded 17939 records

Step 2: Adding cohort IDs...
    Records with cohorts: 24731

Step 3: Calculating margin tiers...

MARGIN TIERS COMPLETE
Total records: 24731

Margin Tier Structure:
  margin_tier_below:   effective_min - step (1 below)
  margin_tier_1:       effective_min_margin
  margin_tier_2:       effective_min + 1*step
  margin_tier_3:       effective_min + 2*step
  margin_tier_4:       effective_min + 3*step
  margin_tier_5:       max_boundary
  margin_tier_above_1: max_boundary + 1*step
  margin_tier_above_2: max_boundary + 2*step
✅ Loaded 24731 margin tier records

FETCHING MARKET DATA
Timestamp: 2026-02-01 13:02:28 Cairo time

Step 1: Fetching raw price data...
  1.1 Ben Soliman prices...


      Loaded 1537 records
  1.2 Marketplace prices...


...

      Loaded 65286 records

Step 2: Joining all market price sources (outer join)...
    Market prices base: 15853 records

Step 3: Adding cohort IDs and supporting data...
    Records after adding cohorts: 23740

Step 4: Processing group-level prices...


/tmp/ipykernel_6519/3245917641.py:139: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


    Records after group processing: 25131

Step 5: Adding WAC and margin data...
    Records with WAC: 24888

Step 6: Filtering by price coverage...
    Records after price coverage filter: 13723

Step 7: Calculating price percentiles...


    Records after price analysis: 12976

Step 8: Converting prices to margins...

MARKET DATA COMPLETE
Total records: 12976
  - With marketplace prices: 12314
  - With scrapped prices: 6320
  - With Ben Soliman prices: 8708
✅ Loaded 12976 market data records

Checking cohort_id availability:
  - df has cohort_id: True
  - df_margin_tiers has cohort_id: True
  - df_market has cohort_id: True

✅ Merged margin tiers: 28390 records
   Margin tier columns added: ['region', 'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm', 'effective_min_margin', 'margin_step', 'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']
✅ Merged market data: 28390 records
   Market columns added: ['region', 'ben_soliman_price', 'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max', 'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped', 'minimum', 'perce

,warehouse_id,product_id,cohort_id,sku,region,brand,cat,current_price,wac_p,new_wac,...,above_market,margin_tier_1,margin_tier_2,margin_tier_3,margin_tier_4,margin_tier_5,margin_tier_above_1,margin_tier_above_2,needs_action,action_reason
269,339,1068,704,ذرة فشار الضحى - 500 جم,NaN,الضحى,بقوليات,72.25,70.015315,70.015315,...,NaN,0.046071,0.055811,0.065550,0.075290,0.085029,0.094769,0.104508,True,growing_performance
441,236,8939,701,ميرندا برتقال - 1.47 لتر,Giza,بيبسي,حاجه ساقعه,182.00,163.908297,163.908297,...,0.219521,0.025512,0.037884,0.050256,0.062628,0.075000,0.087372,0.099744,True,growing_performance
760,797,5851,702,مكرونة كايرو اسباجتى - 400 جم,NaN,كايرو,مكرونة,183.75,178.622441,178.622441,...,NaN,0.028349,0.040012,0.051674,0.063337,0.075000,0.086663,0.098326,True,growing_performance
1727,401,7706,1126,مكرونة بساطة لسان عصفور - 350 جم,Upper Egypt,بساطة,مكرونة,163.50,153.907752,153.907752,...,0.080049,0.037467,0.044870,0.052273,0.059676,0.067079,0.074482,0.081886,True,growing_performance
2639,501,5490,1124,شويبس ليمون نعناع جيب - 240 مل,NaN,شويبس,حاجه ساقعه,279.25,256.370394,266.619852,...,NaN,0.046950,0.064575,0.082199,0.099823,0.117448,0.135072,0.152697,True,wac_increase
3467,962,3673,701,فانتا برتقال - 0.95 لتر,NaN,كوكا كولا,حاجه ساقعه,116.75,106.831013,114.909795,...,NaN,0.056000,0.068291,0.080582,0.092873,0.105164,0.117455,0.129746,True,wac_increase
3684,8,5490,703,شويبس ليمون نعناع جيب - 240 مل,NaN,شويبس,حاجه ساقعه,279.25,256.370394,266.619852,...,NaN,0.049600,0.065478,0.081356,0.097234,0.113112,0.128989,0.144867,True,wac_increase
4912,703,8915,1123,بيبسى - 1.47 لتر,Upper Egypt,بيبسي,حاجه ساقعه,178.25,167.291041,167.291041,...,0.102757,0.035000,0.043656,0.052313,0.060969,0.069626,0.078282,0.086939,True,growing_performance
5361,1,5490,700,شويبس ليمون نعناع جيب - 240 مل,NaN,شويبس,حاجه ساقعه,263.75,256.370394,266.619852,...,NaN,0.046950,0.065213,0.083475,0.101738,0.120000,0.138262,0.156525,True,wac_increase
5798,170,5490,704,شويبس ليمون نعناع جيب - 240 مل,NaN,شويبس,حاجه ساقعه,279.25,256.370394,266.619852,...,NaN,0.046950,0.064241,0.081532,0.098823,0.116113,0.133404,0.150695,True,wac_increase


In [11]:
# =============================================================================
# ACTION LOGIC
# =============================================================================
print("Calculating actions for filtered SKUs...")

# Step 0: Preparation - Calculate normal_refill_3std
df_action['normal_refill_3std'] = df_action['normal_refill'] + 3 * df_action['refill_stddev']

# Initialize output columns
df_action['new_price'] = np.nan
df_action['new_cart_rule'] = np.nan
df_action['price_action'] = 'none'
df_action['cart_rule_action'] = 'none'

# =============================================================================
# HELPER: Calculate all price tiers from margins using wac_p
# =============================================================================
def calculate_price_from_margin(wac, margin):
    """Calculate price from WAC and margin: price = wac / (1 - margin)"""
    if pd.isna(margin) or margin >= 1:
        return np.nan
    return wac / (1 - margin)

# Calculate market prices (using wac_p)
market_margin_cols = ['market_min', 'market_25', 'market_50', 'market_75', 'market_max']
for col in market_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# Calculate margin tier prices (using wac_p)
tier_margin_cols = ['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
                    'margin_tier_above_1', 'margin_tier_above_2']
for col in tier_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# =============================================================================
# HELPER: Find first price above threshold
# =============================================================================
def find_first_price_above(row, threshold, price_cols):
    """Find the first price in price_cols that is above threshold."""
    for col in price_cols:
        price = row.get(col, np.nan)
        if pd.notna(price) and price > threshold:
            return price
    return np.nan

# Define price column order (market first, then tiers)
market_price_cols = ['market_min_price', 'market_25_price', 'market_50_price', 'market_75_price', 'market_max_price']
tier_price_cols = ['margin_tier_1_price', 'margin_tier_2_price', 'margin_tier_3_price', 
                   'margin_tier_4_price', 'margin_tier_5_price', 
                   'margin_tier_above_1_price', 'margin_tier_above_2_price']
all_price_cols = market_price_cols + tier_price_cols

# =============================================================================
# STEP 1: WAC Increase Actions (action_reason = 'both' or 'wac_increase')
# =============================================================================
# Only apply if: current_price < new_wac AND current_price > wac_p
# (price is squeezed between old and new WAC)
wac_increase_mask = (
    df_action['action_reason'].isin(['both', 'wac_increase']) &
    (df_action['current_price'] < df_action['new_wac']) &
    (df_action['current_price'] > df_action['wac_p'])
)

def get_wac_increase_price(row):
    """Find first market/tier price above current_price."""
    # Find first price above current_price (not new_wac)
    new_price = find_first_price_above(row, row['current_price'], all_price_cols)
    return new_price

df_action.loc[wac_increase_mask, 'new_price'] = df_action[wac_increase_mask].apply(get_wac_increase_price, axis=1)
df_action.loc[wac_increase_mask, 'price_action'] = 'wac_increase'

print(f"  WAC increase actions: {wac_increase_mask.sum()} SKUs")
print(f"    (SKUs where wac_p < current_price < new_wac)")

# =============================================================================
# STEP 2: Growing Performance Actions (action_reason = 'growing_performance')
# =============================================================================
growing_mask = df_action['action_reason'] == 'growing_performance'

# Case A: Retailers growing (with or without qty)
rets_growing_mask = growing_mask & (df_action['last_hour_rets_status'] == 'growing')

def get_rets_growing_price(row):
    """Find price for rets growing: market/tier >= current_price * 1.005, or fallback to target margin."""
    min_price = row['current_price'] * 1.005
    
    # Try market prices first
    new_price = find_first_price_above(row, min_price, market_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Try tier prices
    new_price = find_first_price_above(row, min_price, tier_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Fallback: wac_p / (1 - (target_margin * 1.15))
    target_margin = row.get('target_margin', 0)
    if pd.notna(target_margin) and target_margin * 1.15 < 1:
        fallback_price = row['wac_p'] / (1 - (target_margin * 1.15))
        # If fallback price is lower than current price, use current_price * 1.01
        if fallback_price < row['current_price']:
            return row['current_price'] * 1.01
        return fallback_price
    
    return np.nan

df_action.loc[rets_growing_mask, 'new_price'] = df_action[rets_growing_mask].apply(get_rets_growing_price, axis=1)
df_action.loc[rets_growing_mask, 'price_action'] = 'rets_growing'

print(f"  Rets growing actions (price increase): {rets_growing_mask.sum()} SKUs")

# Case B: Qty growing only (rets NOT growing)
qty_only_growing_mask = growing_mask & (df_action['last_hour_qty_status'] == 'growing') & (df_action['last_hour_rets_status'] != 'growing')

def get_qty_growing_cart_rule(row):
    """Calculate new cart rule for qty growing only."""
    # Calculate qty per retailer
    uth_retailers = row.get('uth_retailers', 0)
    if uth_retailers == 0:
        qty_per_retailer = np.inf
    else:
        qty_per_retailer = row['uth_qty'] / uth_retailers
    
    # Get the three options
    option1 = row.get('normal_refill_3std', np.inf)
    option2 = qty_per_retailer
    option3 = row.get('current_cart_rule', np.inf) * 0.85
    
    # Return minimum of the three (excluding inf/nan)
    options = [opt for opt in [option1, option2, option3] if pd.notna(opt) and opt != np.inf]
    if options:
        return min(options)
    return np.nan

df_action.loc[qty_only_growing_mask, 'new_cart_rule'] = df_action[qty_only_growing_mask].apply(get_qty_growing_cart_rule, axis=1)

# Round and apply min/max constraints to new_cart_rule
df_action['new_cart_rule'] = df_action['new_cart_rule'].round()
df_action['new_cart_rule'] = df_action['new_cart_rule'].clip(lower=2, upper=150)

df_action.loc[qty_only_growing_mask, 'cart_rule_action'] = 'qty_growing'

print(f"  Qty growing actions (cart rule change): {qty_only_growing_mask.sum()} SKUs")
print(f"    (Cart rule: rounded, min=2, max=150)")

# =============================================================================
# FINAL STEP: ENFORCE COMMERCIAL MINIMUM PRICE
# =============================================================================
# For all SKUs in df_action, ensure new_price >= commercial_min_price
print(f"\n{'='*60}")
print("ENFORCING COMMERCIAL MINIMUM PRICE")
print(f"{'='*60}")

has_commercial_min = (df_action['commercial_min_price'].notna()) & (df_action['commercial_min_price'] > 0)

# For SKUs with new_price already calculated (growing/wac): MAX(new_price, commercial_min)
has_new_price = df_action['new_price'].notna()
below_min_with_new = has_commercial_min & has_new_price & (df_action['new_price'] < df_action['commercial_min_price'])
df_action.loc[below_min_with_new, 'new_price'] = df_action.loc[below_min_with_new, 'commercial_min_price']
df_action.loc[below_min_with_new, 'price_action'] = df_action.loc[below_min_with_new, 'price_action'].astype(str) + '_commercial_enforced'

# For SKUs without new_price (only commercial min violation): new_price = commercial_min
no_new_price = df_action['new_price'].isna()
below_min_only = has_commercial_min & no_new_price & (df_action['current_price'] < df_action['commercial_min_price'])
df_action.loc[below_min_only, 'new_price'] = df_action.loc[below_min_only, 'commercial_min_price']
df_action.loc[below_min_only, 'price_action'] = 'commercial_min_enforced'

print(f"  SKUs with commercial min: {has_commercial_min.sum()}")
print(f"  Growing SKUs with commercial min applied: {below_min_with_new.sum()}")
print(f"  SKUs only needing commercial min: {below_min_only.sum()}")
print(f"  Total commercial min enforcements: {below_min_with_new.sum() + below_min_only.sum()}")

# =============================================================================
# SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("ACTION SUMMARY")
print(f"{'='*60}")
print(f"\nPrice Actions:")
print(df_action['price_action'].value_counts().to_string())
print(f"\nCart Rule Actions:")
print(df_action['cart_rule_action'].value_counts().to_string())

# Show SKUs with new prices
price_changes = df_action[df_action['new_price'].notna()]
print(f"\nSKUs with new price: {len(price_changes)}")
if len(price_changes) > 0:
    print(f"  Avg price change: {((price_changes['new_price'] - price_changes['current_price']) / price_changes['current_price']).mean()*100:.1f}%")

# Show SKUs with new cart rules
cart_changes = df_action[df_action['new_cart_rule'].notna()]
print(f"\nSKUs with new cart rule: {len(cart_changes)}")
if len(cart_changes) > 0:
    print(f"  Avg cart rule change: {((cart_changes['new_cart_rule'] - cart_changes['current_cart_rule']) / cart_changes['current_cart_rule']).mean()*100:.1f}%")

# Display sample
print(f"\n{'='*60}")
print("SAMPLE OUTPUT")
print(f"{'='*60}")
output_cols = ['sku', 'current_price', 'new_price', 'current_cart_rule', 'new_cart_rule', 
               'price_action', 'cart_rule_action', 'action_reason']
output_cols = [c for c in output_cols if c in df_action.columns]
display(df_action[df_action['new_price'].notna() | df_action['new_cart_rule'].notna()][output_cols].head(15))


Calculating actions for filtered SKUs...


  WAC increase actions: 3 SKUs
    (SKUs where wac_p < current_price < new_wac)
  Rets growing actions (price increase): 19 SKUs
  Qty growing actions (cart rule change): 2 SKUs
    (Cart rule: rounded, min=2, max=150)

ENFORCING COMMERCIAL MINIMUM PRICE
  SKUs with commercial min: 1
  Growing SKUs with commercial min applied: 0
  SKUs only needing commercial min: 1
  Total commercial min enforcements: 1

ACTION SUMMARY

Price Actions:
price_action
rets_growing               19
none                       14
wac_increase                3
commercial_min_enforced     1

Cart Rule Actions:
cart_rule_action
none           35
qty_growing     2

SKUs with new price: 23
  Avg price change: 3.0%

SKUs with new cart rule: 2
  Avg cart rule change: -85.5%

SAMPLE OUTPUT


,sku,current_price,new_price,current_cart_rule,new_cart_rule,price_action,cart_rule_action,action_reason
269,ذرة فشار الضحى - 500 جم,72.25,73.396776,25,NaN,rets_growing,none,growing_performance
441,ميرندا برتقال - 1.47 لتر,182.00,185.000000,50,NaN,rets_growing,none,growing_performance
760,مكرونة كايرو اسباجتى - 400 جم,183.75,186.067294,10,NaN,rets_growing,none,growing_performance
1727,مكرونة بساطة لسان عصفور - 350 جم,163.50,165.000000,194,NaN,rets_growing,none,growing_performance
4912,بيبسى - 1.47 لتر,178.25,NaN,22,6.0,none,qty_growing,growing_performance
5361,شويبس ليمون نعناع جيب - 240 مل,263.75,269.000000,34,NaN,wac_increase,none,wac_increase
9004,سوفى ليلية ماكسى بالمسك طويل جدا جدا 36 سم عرض...,82.25,82.869749,28,NaN,rets_growing,none,growing_performance
9698,سائل اطباق وفير بلس ليمون اصفر - 725 جم,190.25,199.973333,9,NaN,rets_growing,none,growing_performance
10115,شوكولاتة جلاكسى فلوتس كريم - 11.25 جم,100.00,104.451298,9,NaN,rets_growing,none,growing_performance
12218,صابون لوكس بشرة نضرة - 115 جم,95.75,96.427723,9,NaN,rets_growing,none,growing_performance


In [12]:
# =============================================================================
# PUSH CONFIGURATION
# =============================================================================
# Mode: 'testing' = prepare files only, 'live' = actually upload to API
PUSH_MODE = 'live'  # Change to 'live' when ready to push

print(f"\n{'='*60}")
print(f"PUSH MODE: {PUSH_MODE.upper()}")
print(f"{'='*60}")
if PUSH_MODE == 'testing':
    print("⚠️ Testing mode - files will be prepared but NOT uploaded")
else:
    print("🚀 Live mode - files will be uploaded to MaxAB API")



PUSH MODE: LIVE
🚀 Live mode - files will be uploaded to MaxAB API


In [13]:
df_action.to_excel("check.xlsx")

In [14]:
# =============================================================================
# IMPORT PUSH HANDLERS
# =============================================================================
%run push_cart_rules_handler.ipynb
%run push_prices_handler.ipynb

print("✅ Push handlers loaded")


Push Cart Rules Handler loaded at 2026-02-01 13:03:17 Cairo time


✓ API credentials loaded successfully


Push Prices Handler loaded at 2026-02-01 13:03:18 Cairo time


✓ API credentials loaded successfully


✓ Google Sheets client initialized
✅ Push handlers loaded


In [15]:
# =============================================================================
# PREPARE DATA FOR PUSH
# =============================================================================
# Get packing units for push handlers
pus = get_packing_units()
print(f"✅ Loaded {len(pus)} packing units")

# Prepare df_action with required columns for push functions
# Prices need: product_id, sku, new_price, warehouse_id, cohort_id, stocks, current_price
# Cart rules need: product_id, sku, new_cart_rule, warehouse_id, cohort_id, stocks, current_cart_rule

# Rename stocks column if needed
if 'current_stocks' in df_action.columns and 'stocks' not in df_action.columns:
    df_action['stocks'] = df_action['current_stocks']

# Summary of what will be pushed
price_changes = df_action[df_action['new_price'].notna() & (df_action['new_price'] != df_action['current_price'])]
cart_changes = df_action[df_action['new_cart_rule'].notna() & (df_action['new_cart_rule'] != df_action['current_cart_rule'])]

print(f"\n📊 Push Summary:")
print(f"  Price changes: {len(price_changes)} SKUs")
print(f"  Cart rule changes: {len(cart_changes)} SKUs")
print(f"  Mode: {PUSH_MODE}")


Fetching packing_units ...


  Loaded 34962 records
✅ Loaded 34962 packing units

📊 Push Summary:
  Price changes: 23 SKUs
  Cart rule changes: 2 SKUs
  Mode: live


In [16]:
# =============================================================================
# STEP 1: PUSH CART RULES
# =============================================================================
# Push cart rules first - if any cohorts fail, we'll skip them for prices too
print(f"\n{'='*60}")
print("STEP 1: PUSH CART RULES")
print(f"{'='*60}")

cart_result = push_cart_rules(df_action, pus, source_module='module_4', mode=PUSH_MODE)

# Track failed cohorts to skip in price push
failed_cohorts = cart_result.get('failed_cohorts', [])

print(f"\n📊 Cart Rules Push Result:")
print(f"  Total received: {cart_result['total_received']}")
print(f"  Cart rule changes: {cart_result['cart_rule_changes']}")
print(f"  Pushed: {cart_result['pushed']}")
print(f"  Skipped: {cart_result['skipped']}")
print(f"  Failed: {cart_result['failed']}")
print(f"  Mode: {cart_result['mode']}")

if failed_cohorts:
    print(f"  ⚠️ Failed cohorts: {failed_cohorts}")



STEP 1: PUSH CART RULES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API

PUSH CART RULES - Source: module_4
Total received: 37
Cart rule changes to push: 2
Skipped (no change): 35

Cart rule changes summary:
  Increases: 0
  Decreases: 2

📋 Prepared 2 packing unit cart rules

Processing cohort: 701
  Saved: uploads/module_4_cart_rules_701.xlsx (1 rows)


  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 266.73it/s]

  Uploading...


...

  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 255.86it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 2
Total failed: 0

📊 Cart Rules Push Result:
  Total received: 37
  Cart rule changes: 2
  Pushed: 2
  Skipped: 35
  Failed: 0
  Mode: live


In [17]:
# =============================================================================
# STEP 2: PUSH PRICES (skip failed cohorts from cart rules)
# =============================================================================
print(f"\n{'='*60}")
print("STEP 2: PUSH PRICES")
print(f"{'='*60}")

# Push prices, skipping any cohorts that failed during cart rules push
push_result = push_prices(df_action, pus, source_module='module_4', mode=PUSH_MODE, skip_cohorts=failed_cohorts)

print(f"\n📊 Prices Push Result:")
print(f"  Total received: {push_result['total_received']}")
print(f"  Price changes: {push_result['price_changes']}")
print(f"  Pushed: {push_result['pushed']}")
print(f"  Skipped: {push_result['skipped']}")
print(f"  Failed: {push_result['failed']}")
print(f"  Mode: {push_result['mode']}")

if push_result.get('skipped_cohorts'):
    print(f"  ⚠️ Skipped cohorts: {push_result['skipped_cohorts']}")



STEP 2: PUSH PRICES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API
Loading disable_pu_visibility from Google Sheets...


  ✓ Loaded 88 products to disable min PU visibility

PUSH PRICES - Source: module_4
Total received: 37
Price changes to push: 23
Skipped (no change): 14

Price changes summary:
  Increases: 23
  Decreases: 0

📋 Prepared 32 packing unit prices

Processing cohort: 704
  Saved: uploads/module_4_704.xlsx (10 rows)


  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 162.76it/s]

...

  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 222.23it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 32
Total failed: 0

📊 Prices Push Result:
  Total received: 37
  Price changes: 23
  Pushed: 32
  Skipped: 14
  Failed: 0
  Mode: live


In [18]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("MODULE 4 - HOURLY UPDATES COMPLETE")
print(f"{'='*60}")
print(f"\n📅 Timestamp: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Mode: {PUSH_MODE.upper()}")

print(f"\n📊 ACTIONS TAKEN:")
print(f"  Total SKUs analyzed: {len(df)}")
print(f"  SKUs requiring action: {len(df_action)}")

# Price actions breakdown
print(f"\n💰 PRICE ACTIONS:")
if 'price_action' in df_action.columns:
    price_actions = df_action[df_action['price_action'] != 'none']
    print(f"  Total price changes: {len(price_actions)}")
    for action_type in df_action['price_action'].unique():
        if action_type != 'none':
            count = len(df_action[df_action['price_action'] == action_type])
            print(f"    - {action_type}: {count}")
else:
    print(f"  Total price changes: 0")

# Cart rule actions breakdown  
print(f"\n🛒 CART RULE ACTIONS:")
if 'cart_rule_action' in df_action.columns:
    cart_actions = df_action[df_action['cart_rule_action'] != 'none']
    print(f"  Total cart rule changes: {len(cart_actions)}")
    for action_type in df_action['cart_rule_action'].unique():
        if action_type != 'none':
            count = len(df_action[df_action['cart_rule_action'] == action_type])
            print(f"    - {action_type}: {count}")
else:
    print(f"  Total cart rule changes: 0")

# Push results
print(f"\n📤 PUSH RESULTS:")
print(f"  Cart Rules - Pushed: {cart_result.get('pushed', 0)}, Failed: {cart_result.get('failed', 0)}")
print(f"  Prices - Pushed: {push_result.get('pushed', 0)}, Failed: {push_result.get('failed', 0)}")

if PUSH_MODE == 'testing':
    print(f"\n⚠️ TESTING MODE - No changes were actually pushed to production!")
    print(f"   Change PUSH_MODE to 'live' to execute actual pushes.")
else:
    print(f"\n✅ LIVE MODE - Changes have been pushed to production!")



MODULE 4 - HOURLY UPDATES COMPLETE

📅 Timestamp: 2026-02-01 13:04:01
🔧 Mode: LIVE

📊 ACTIONS TAKEN:
  Total SKUs analyzed: 28390
  SKUs requiring action: 37

💰 PRICE ACTIONS:
  Total price changes: 23
    - rets_growing: 19
    - wac_increase: 3
    - commercial_min_enforced: 1

🛒 CART RULE ACTIONS:
  Total cart rule changes: 2
    - qty_growing: 2

📤 PUSH RESULTS:
  Cart Rules - Pushed: 2, Failed: 0
  Prices - Pushed: 32, Failed: 0

✅ LIVE MODE - Changes have been pushed to production!


In [19]:
# =============================================================================
# SAMPLE OUTPUT - Current Status
# =============================================================================
# Show sample of data with all calculated fields

sample_cols = [
    'warehouse_id', 'product_id', 'sku','wac1','wac_p','new_wac',
    # P80/P70 benchmarks and std
    'p80_daily_240d', 'std_daily_240d', 'p70_daily_retailers_240d', 'std_daily_retailers_240d',
    # Current cart rule
    'current_cart_rule',
    # UTH performance (with std thresholds)
    'uth_qty', 'uth_qty_target', 'uth_qty_std', 'uth_qty_status',
    'uth_retailers', 'uth_rets_target', 'uth_rets_std', 'uth_rets_status',
    # Last hour performance (with std thresholds)
    'last_hour_qty', 'last_hour_qty_target', 'last_hour_qty_std', 'last_hour_qty_status',
    'last_hour_retailers', 'last_hour_rets_target', 'last_hour_rets_std', 'last_hour_rets_status'
]

# Filter to columns that exist
sample_cols = [c for c in sample_cols if c in df.columns]

print(f"\n{'='*60}")
print("SAMPLE DATA (First 10 rows with UTH > 0)")
print(f"{'='*60}")
sample = df[df['uth_qty'] > 0][sample_cols].head(10)
display(sample)



SAMPLE DATA (First 10 rows with UTH > 0)


,warehouse_id,product_id,sku,wac1,wac_p,new_wac,p80_daily_240d,std_daily_240d,p70_daily_retailers_240d,std_daily_retailers_240d,...,uth_rets_std,uth_rets_status,last_hour_qty,last_hour_qty_target,last_hour_qty_std,last_hour_qty_status,last_hour_retailers,last_hour_rets_target,last_hour_rets_std,last_hour_rets_status
2,236,5946,عصير سن توب تفاح - 250 مل,264.019604,234.977448,234.977448,12.0,6.640988,8.000,3.337612,...,1.179224,on_track,0.0,0.430318,0.238145,dropping,0.0,0.295713,0.123372,dropping
3,236,10816,بليزو ويفر محشو بالشوكولاتة مغطي بالكراميل و ا...,42.845315,42.845315,42.845315,12.2,8.405438,2.000,1.493718,...,0.547058,on_track,0.0,0.372737,0.256805,dropping,0.0,0.055839,0.041704,dropping
5,339,10452,هارفست فول دندراوى خصم %15 - 400 جم,17.880000,16.270800,16.270800,24.0,10.606602,2.000,0.707107,...,0.203406,on_track,0.0,0.703430,0.310875,dropping,0.0,0.066284,0.023435,dropping
6,170,11965,زيت كريستال الممتاز خليط - 2.5 لتر,674.000000,653.787688,653.787688,5.0,0.994739,2.000,0.769338,...,0.221354,growing,0.0,0.184344,0.036675,dropping,0.0,0.074364,0.028606,dropping
10,8,416,عصير جهينة جوافة - 1 لتر,361.203686,324.180308,324.180308,6.0,2.653062,4.799,2.213043,...,0.577148,on_track,0.0,0.235437,0.104105,dropping,0.0,0.195158,0.089997,dropping
11,401,336,طحينة البوادى - 125 جم,28.028213,25.594289,25.594289,30.0,19.486031,2.000,1.312536,...,0.367191,on_track,0.0,0.928386,0.603019,dropping,0.0,0.071593,0.046984,dropping
12,1,10540,ابو عوف جولد قهوة سريعة الذوبان - 24 كيس,107.000477,92.020410,92.020410,12.6,5.791494,6.000,2.434411,...,0.794972,growing,2.0,0.434555,0.199740,growing,1.0,0.221285,0.089783,growing
13,236,159,مرقة دجاج كنور فاين فودز شريط - 8 جم,43.968052,40.265864,40.265864,302.4,506.930033,17.000,8.587473,...,2.654769,on_track,0.0,10.836564,18.165939,dropping,0.0,0.642440,0.324526,dropping
14,1,220,بن عبد المعبود سادة فاتح - 100 جم,549.772747,528.952949,528.952949,7.0,3.880291,5.000,2.591484,...,0.846264,on_track,0.0,0.241420,0.133825,dropping,0.0,0.184404,0.095576,dropping
15,170,12212,مناديل سمايل سحب 550 منديل,233.198401,233.198401,233.198401,8.0,4.706860,3.000,1.576861,...,0.441041,on_track,0.0,0.272358,0.160244,dropping,0.0,0.107285,0.056391,dropping


In [20]:
# =============================================================================
# UPLOAD RESULTS TO SNOWFLAKE AND SEND SLACK NOTIFICATION
# =============================================================================

from common_functions import upload_dataframe_to_snowflake, send_text_slack

# Add created_at as TIMESTAMP (module runs hourly)
df_action['created_at'] = datetime.now(CAIRO_TZ).replace(second=0, microsecond=0)
df_action = df_action.drop(columns='commercial_min_price')
# Upload to Snowflake
print(f"\n{'='*60}")
print("UPLOADING RESULTS TO SNOWFLAKE")
print(f"{'='*60}")

upload_status = upload_dataframe_to_snowflake(
    "Egypt", 
    df_action, 
    "MATERIALIZED_VIEWS", 
    "pricing_hourly_push", 
    "append", 
    auto_create_table=True, 
    conn=None
)

# Prepare status variables
prices_pushed = push_result.get('pushed', 0) if 'push_result' in dir() else 0
prices_failed = push_result.get('failed', 0) if 'push_result' in dir() else 0
cart_rules_pushed = cart_result.get('pushed', 0) if 'cart_result' in dir() else 0
cart_rules_failed = cart_result.get('failed', 0) if 'cart_result' in dir() else 0

# Count price and cart rule actions
price_changes = len(df_action[df_action['price_action'] != 'none']) if 'price_action' in df_action.columns else 0
cart_changes = len(df_action[df_action['cart_rule_action'] != 'none']) if 'cart_rule_action' in df_action.columns else 0

if upload_status:
    slack_message = f"""✅ *Module 4 - Hourly Updates Completed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Hour: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
🔧 Mode: {PUSH_MODE.upper()}

📊 *Results:*
• Total SKUs analyzed: {len(df):,}
• SKUs requiring action: {len(df_action):,}
• Price changes: {price_changes:,}
• Cart rule changes: {cart_changes:,}

📤 *Push Status:*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed

🗄️ Results uploaded to: MATERIALIZED_VIEWS.pricing_hourly_push"""
    
    send_text_slack('new-pricing-logic', slack_message)
    print("✅ Slack notification sent!")
    print(f"✅ {len(df_action)} records uploaded to Snowflake")
else:
    error_message = f"""❌ *Module 4 - Hourly Updates Failed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Hour: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
⚠️ Upload to Snowflake failed - please check logs

📤 *Push Status (before upload failure):*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed"""
    
    send_text_slack('new-pricing-logic', error_message)
    print("❌ Error notification sent to Slack!")



UPLOADING RESULTS TO SNOWFLAKE


/home/ec2-user/service_account_key.json


/home/ec2-user/SageMaker/Pricing Runs/Prediction_Scripts_2/Happy_hour/git/Mustafa/Pricing Logic/modules/../common_functions.py:698: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, _ = write_pandas(


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://docs.slack.dev/tools/python-slack-sdk/v3-migration/
  warnings.warn(message)


/home/ec2-user/service_account_key.json


Message Sent
✅ Slack notification sent!
✅ 37 records uploaded to Snowflake
